# Course Project - Object Detection Using CNNs in RADAR Signal Imagery  

Course Name: EE904 - Deep Learning For Wireless Communication   
Professor: Prof. Tushar Sandhan  
Academic Quarter: Q3-2023-2024  
Author and Student Name: Venkateswar Reddy Melachervu  
Roll No: 23156022  
Email: vmela23@iitk.ac.in  
  
**Project Description**  
The principal objective of this project is to classify, using deep neural network, a given/unseen RADAR signal imagery obtained from the RADAR scan of a wall accurately into:  
>***Empty***: *No person behind the wall*  
>***Still***: *A person is present behind the wall in stand-still position*  
>***Walking***: *A person is walking behind the wall*  

This scenario finds crucial application in military endeavors and surveillance.

**Training Dataset**  
Training data set of RADAR scan imagery taken through a concrete wall is provided as input to this project and was made available at [this link](https://drive.google.com/drive/folders/1g5NS45zaoxXOran9CYM8J49tTyhZ_9Hd). The dataset contains train folder with sample imagery for training the deep network and test folder with test magery to evaluate the performance of the developed network model.  

**Program Tasks**  
The main task is to investigate/classify the RADAR signal imagery accurately for the object behind the wall - empty, person standing still or walking. Additionally, this project investigates higher accuracy CNN model, using trial and error method, for this classification while addressing the below requirements of the project assignment.
1.   Plots 5 random images from each class on a single row and details an intuitive human analysis of imagery patterns to manually classify any new sample into one of the three classes - *Empty, Still, Walking*
2.   Selects 10 samples randomly from each class, mixes them all up for manual classification and creates a report on the correct classification and difficulties faced for in-correct classification.
3.   Uses non-CNN MLP NN for training and testing and creates a report on accuracy, training time, accuracy graphs
4.   Designs and uses a CNN with 1 hidden layer and trains it on provided training data and creates a report on accuracy vs epoch (graphs), model details, total number of learnable parameters, training time per epoch etc.
5.   Evaluates the above trained CNN model with the test data set and creates a report on accuracy and plots confusion matrix.
6.   Designs CNN with multiple hidden layers and creates a report on accuracy vs epoch (graphs), model details, total number of learnable parameters, training time per epoch etc.
7.   Changes number of hidden layers, number of kernels and creates a report on accuracy vs epoch (graphs), model details, total number of learnable parameters, training time per epoch etc.
8.   Uses pre-trained ResNet model and creates a report on accuracy vs epoch (graphs), model details, total number of learnable parameters, training time per epoch etc.

**Dependencies**  
This project leverages the below libraries:  
>1  Tensorflow  
>2  Keras  
>3  ResNet  
>4  gdown  
>5  PIL  
>6  numpy  
>7  google drive for training test data set etc.

**References**  

**Additional Information**  


In [ ]:
# let's do the imports and initializations and installs
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import time
import numpy as np
import os

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# configure the train and test data directory
train_dir = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/train'
train_dir_empty = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/train/empty'
train_dir_still = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/train/still'
train_dir_walking = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/train/walking'
val_dir = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/val/'
tests_dir = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/tests/'
tests_dir_empty = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/tests/empty'
tests_dir_still = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/tests/still'
tests_dir_walking = '/content/drive/MyDrive/EE-904-GPR-CNN-Project/dataset/tests/walking'

# load the training samples from training data dir
import pathlib
data_dir = pathlib.Path(train_dir)
tests_dir = pathlib.Path(tests_dir)
val_dir = pathlib.Path(val_dir)

In [ ]:
# sanity checks for test/validation folder contents
files = os.listdir(train_dir)
print('Files/dirs in '+ str(train_dir) + ':' + str(files))
files = os.listdir(tests_dir)
print('Files/dirs in '+ str(tests_dir) + ':' + str(files))

# sanity check for training data
images = list(data_dir.glob('*/*.jpg'))
image_count = len(images)
print('Count of images in ' + str(data_dir) + ':' + str(image_count))
empty = list(data_dir.glob('empty/*'))
empty_image_count = len(empty)
print('\"empty\" class training image sample count' + ':' + str(empty_image_count))
print('Displaying a training image in \"empty\" class...')
PIL.Image.open(str(empty[0]))

# sanity check for training data
still = list(data_dir.glob('still/*'))
still_image_count = len(still)
print('\"still\" class training image sample count' + ':' + str(still_image_count))
print('Displaying a training image in \"still\" class...')
PIL.Image.open(str(still[0]))

# sanity check for training data
walking = list(data_dir.glob('walking/*'))
walking_image_count = len(still)
print('\"walking\" class training image sample count' + ':' + str(walking_image_count))
print('Displaying a training image in \"walking\" class...')
PIL.Image.open(str(walking[0]))

# sanity check for tests data
images = list(tests_dir.glob('*/*.jpg'))
image_count = len(images)
print('Count of all images in ' + str(tests_dir) + ':' + str(image_count))
empty = list(tests_dir.glob('empty/*'))
empty_image_count = len(empty)
print('\"empty\" class test image sample count' + ':' + str(empty_image_count))
print('Displaying a test image in \"empty\" class...')
PIL.Image.open(str(empty[0]))

# sanity check for validation data
images = list(val_dir.glob('*/*.jpg'))
image_count = len(images)
print('Count of images in ' + str(val_dir) + ':' + str(image_count))
empty = list(val_dir.glob('empty/*'))
empty_image_count = len(empty)
print('\"empty\" class validation image sample count' + ':' + str(empty_image_count))
print('Displaying a validation image in \"empty\" class...')
PIL.Image.open(str(empty[0]))

# Displaying 5 random images of Empty class in a row from training data set
from PIL import Image
import os
import random
import matplotlib.pyplot as plt

def display_random_images(directory, image_width, image_height):
    plt.figure(figsize=(image_width, image_height))
    plt.suptitle('5 Empty Class Random Training Samples', fontsize=12)
    image_files = os.listdir(directory)
    random.shuffle(image_files)
    for i in range(5):
        img_path = os.path.join(directory, image_files[i])
        img = Image.open(img_path)
        plt.subplot(1, 5, i+1)
        plt.imshow(img, cmap='gray')  # Specify colormap as 'gray' for grayscale images
        plt.title(image_files[i], fontsize=10)
        plt.axis('off')
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

# Display from "Empty" training directory
directory = train_dir_empty
image_width = 8  # Width of each image in inches
image_height = 7.0  # Height of each image in inches
display_random_images(directory, image_width, image_height)

# Displaying 5 random images of Still class in a row from training data set
from PIL import Image
import os
import random
import matplotlib.pyplot as plt

def display_random_images(directory, image_width, image_height):
    plt.figure(figsize=(image_width, image_height))
    plt.suptitle('5 Still Class Random Training Samples', fontsize=12)
    image_files = os.listdir(directory)
    random.shuffle(image_files)
    for i in range(5):
        img_path = os.path.join(directory, image_files[i])
        img = Image.open(img_path)
        plt.subplot(1, 5, i+1)
        plt.imshow(img, cmap='gray')  # Specify colormap as 'gray' for grayscale images
        plt.title(image_files[i], fontsize=10)
        plt.axis('off')
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

# Display from "Still" training directory
directory = train_dir_still
image_width = 8  # Width of each image in inches
image_height = 7.0  # Height of each image in inches
display_random_images(directory, image_width, image_height)

# Displaying 5 random images of Walking class in a row from training data set
from PIL import Image
import os
import random
import matplotlib.pyplot as plt

def display_random_images(directory, image_width, image_height):
    plt.figure(figsize=(image_width, image_height))
    plt.suptitle('5 Walking Class Random Training Samples', fontsize=12)
    image_files = os.listdir(directory)
    random.shuffle(image_files)
    for i in range(5):
        img_path = os.path.join(directory, image_files[i])
        img = Image.open(img_path)
        plt.subplot(1, 5, i+1)
        plt.imshow(img, cmap='gray')  # Specify colormap as 'gray' for grayscale images
        plt.title(image_files[i], fontsize=10)
        plt.axis('off')
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

# Display from "Walking" training directory
directory = train_dir_walking
image_width = 8  # Width of each image in inches
image_height = 7.0  # Height of each image in inches
display_random_images(directory, image_width, image_height)

In [2]:
# define reusable data pipeline function
def setup_data_pipeline(train_dir, test_dir, validation_split=0.2, batch_size=32, img_height=768, img_width=32, seed=12081970):
  # Training dataset configuration
  print('Preparing training data set...')
  print('Image size used for training ' + str(img_width) + 'x' + str(img_height))
  print('Validation split of the training samples ' + str(validation_split*100) + '%')
  train_ds = tf.keras.utils.image_dataset_from_directory(
      train_dir,
      validation_split=validation_split,
      subset="training",
      seed=seed,
      image_size=(img_height, img_width),
      batch_size=batch_size
  )
  print('Done - Preparing training data set.')

  # Validation dataset configuration
  print('Preparing validation data set...')
  val_ds = tf.keras.utils.image_dataset_from_directory(
      train_dir,
      validation_split=validation_split,
      subset="validation",
      seed=seed,
      image_size=(img_height, img_width),
      batch_size=batch_size
  )
  print('Done - Preparing validation data set.')

  # Test dataset configuration
  print('Preparing test data set...')
  test_ds = tf.keras.utils.image_dataset_from_directory(
      test_dir,
      seed=seed,
      image_size=(img_height, img_width),
      batch_size=batch_size
  )
  print('Done - Preparing test data set.')

  # List of inferred class names
  class_names = train_ds.class_names
  print('List of all training classes:', class_names)

  # training batch set info
  print('Training set batches information:')
  for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)

  # perforance settings - buffered prefetching
  print('Setting buffered prefetching the data/images for better execution performance of the program...')
  # Performance settings - buffered prefetching
  AUTOTUNE = tf.data.AUTOTUNE
  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
  print('Done - Setting buffered pre-fetch.')

  return train_ds, val_ds, test_ds, class_names

In [ ]:
# tensorflow dashboard for gradient descent analytics
!pip install ngrok
!pip install pyngrok

import ngrok
from pyngrok import ngrok

# Set Ngrok authtoken
ngrok.set_auth_token("2dgXfPL1pbdqUcPXyS5axzhmnrB_4Ux9eRzJVVyUPJD12NEqy")

# Start ngrok
# Establish a tunnel to the TensorBoard port
public_url = ngrok.connect(addr="6006", bind_tls=True).public_url
print("TensorBoard URL:", public_url)
ngrok_tunnel = ngrok.connect(addr="6006", bind_tls=True)
# Print the public URL where TensorBoard is exposed
print("TensorBoard URL:", ngrok_tunnel.public_url)
%load_ext tensorboard
log_dir = "logs/fit"
%tensorboard --logdir $log_dir --bind_all

In [ ]:
# Normal MLP NN model without any convolution operation
# hyper paramters definition
batch_size = 32 # to run back propagation epoch
img_height = 768
img_width = 32
validation_split=0.2
seed=12081970
epochs = 10

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = len(class_names)
input_shape = (img_height,img_width, 3)

# design the model
normal_mlp_nn_model_1 = Sequential([
  layers.Rescaling(1./255, input_shape=input_shape),
  Flatten(input_shape=input_shape),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dense(num_classes, activation='softmax')
])
normal_mlp_nn_model_1._name = 'Normal_MLP_NN_Without_Convolution_And_Original_Image_Resolution_Training_1'

# compile the model
normal_mlp_nn_model_1.compile(
  # Adam optimizer
  optimizer='adam',
  # # Loss function for classification tasks
  loss='sparse_categorical_crossentropy',
  # Metric to monitor during training (e.g., accuracy)
  metrics=['accuracy'])

# let's print the summary of the cnn basic model we built
normal_mlp_nn_model_1.summary()
# Print activation functions
print("Activation Functions:")
for layer in normal_mlp_nn_model_1.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# let's train the model - 10 epochs

import time
start_time = time.time()
history = normal_mlp_nn_model_1.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs
)
end_time = time.time()
training_time = end_time - start_time
print("Training Time:", training_time, "seconds")

# let's plot all graphs to visually understand and evaluate the performance for nomal MLP model
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# training time per epoch graph
import matplotlib.pyplot as plt

# Calculate training time per epoch
training_time_per_epoch = (end_time - start_time) / epochs

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Normal MLP NN model without any convolution operation - non-original size image training
# hyper paramters definition
batch_size = 32 # to run back propagation epoch
# img_height = 768
# img_width = 32
img_height = 50
img_width = 5
validation_split=0.2
seed=12081970
epochs = 10

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = len(class_names)
input_shape = (img_height,img_width, 3)

# design the model
normal_mlp_nn_model_2 = Sequential([
  layers.Rescaling(1./255, input_shape=input_shape),
  Flatten(input_shape=input_shape),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dense(num_classes, activation='softmax')
])
normal_mlp_nn_model_2._name = 'Normal_MLP_NN_Without_Convolution_And_Non_Original_Image_Resolution_Training_2'

# compile the model
normal_mlp_nn_model_2.compile(
  # Adam optimizer
  optimizer='adam',
  # # Loss function for classification tasks
  loss='sparse_categorical_crossentropy',
  # Metric to monitor during training (e.g., accuracy)
  metrics=['accuracy'])

# let's print the summary of the cnn basic model we built
normal_mlp_nn_model_2.summary()
# Print activation functions
print("Activation Functions:")
for layer in normal_mlp_nn_model_2.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# let's train the model - 10 epochs
epochs = 10
import time
start_time = time.time()
history = normal_mlp_nn_model_2.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs
)
end_time = time.time()
training_time = end_time - start_time
print("Training Time:", training_time, "seconds")

# let's plot all graphs to visually understand and evaluate the performance
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# training time per epoch graph
import matplotlib.pyplot as plt

# Calculate training time per epoch
training_time_per_epoch = (end_time - start_time) / epochs

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# CNN with one hidden layer and RELU activation
batch_size = 32 # to run back propagation epoch
img_height = 768
img_width = 32
validation_split=0.2
seed=12081970
epochs = 10

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = len(class_names)
input_shape = (img_height,img_width, 3)
cnn_model_with_1hl = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
cnn_model_with_1hl._name = 'CNN_with_One_Hidden_Layer_RELU_Activation_with_Original_Image_Trainin_1'

# Compile the model
cnn_model_with_1hl.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Model details
cnn_model_with_1hl.summary()
# Print activation functions
print("Activation Functions:")
for layer in cnn_model_with_1hl.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# Train the model
start_time = time.time()
history = cnn_model_with_1hl.fit(train_ds, validation_data=val_ds, epochs=epochs)
end_time = time.time()
training_time_per_epoch = (end_time - start_time) / epochs
print("Training Time Per Epoch:", training_time_per_epoch, "seconds")

# Convert accuracy values to percentage
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# Plot accuracy versus epoch in percentage
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate the cnn_model_with_one_hidden_layer on the test data available in val directory dataset
print('Evaluating the model accuracy with test folder data...')
test_loss, test_accuracy = cnn_model_with_1hl.evaluate(test_ds)

# Plot accuracy in percentage
print("Test Accuracy:", test_accuracy * 100, "%")

# Predict labels for the test dataset
test_predictions = cnn_model_with_1hl.predict(test_ds)
test_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# CNN with one hidden layer and RELU activation with non-original image size
batch_size = 32 # to run back propagation epoch
img_height = 190
img_width = 16
validation_split=0.2
seed=12081970
epochs = 10

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = len(class_names)
input_shape = (img_height,img_width, 3)
cnn_model_with_1hl_2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
cnn_model_with_1hl_2._name = 'CNN_with_One_Hidden_Layer_RELU_Activation_with_Non_Original_Image_Training_2'

# Compile the model
cnn_model_with_1hl_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Model details
cnn_model_with_1hl_2.summary()
# Print activation functions
print("Activation Functions:")
for layer in cnn_model_with_1hl_2.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# Train the model
start_time = time.time()
history = cnn_model_with_1hl_2.fit(train_ds, validation_data=val_ds, epochs=epochs)
end_time = time.time()
training_time_per_epoch = (end_time - start_time) / epochs
print("Training Time Per Epoch:", training_time_per_epoch, "seconds")

# Convert accuracy values to percentage
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# Plot accuracy versus epoch in percentage
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate the cnn_model_with_one_hidden_layer on the test data available in val directory dataset
print('Evaluating the model accuracy with test folder data...')
test_loss, test_accuracy = cnn_model_with_1hl_2.evaluate(test_ds)

# Plot accuracy in percentage
print("Test Accuracy:", test_accuracy * 100, "%")

# Predict labels for the test dataset
test_predictions = cnn_model_with_1hl_2.predict(test_ds)
test_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# CNN with multiple hidden layer and RELU activation, 32 filter kernels and visual plots of gradient descent
batch_size = 32 # to run back propagation epoch
img_height = 768
img_width = 32
validation_split=0.2
seed=12081970
epochs = 10

from tensorflow.keras.callbacks import TensorBoard
import datetime

# Let's define log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Let's create TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = len(class_names)
input_shape = (img_height,img_width, 3)

cnn_model_with_multiple_hidden_layers_1 = models.Sequential([
    layers.Rescaling(1./255, input_shape=input_shape),
    layers.Conv2D(32, 3, padding='same', activation='relu'), # filters 32, kernel size 3
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
cnn_model_with_multiple_hidden_layers_1._name = 'CNN_with_Multiple_Hidden_Layers_RELU_Activation_with_Original_Image_Training_1'

# Compile the model
cnn_model_with_multiple_hidden_layers_1.compile(optimizer='adam',
                                  loss='sparse_categorical_crossentropy',
                                  metrics=['accuracy'])
# Model details
cnn_model_with_multiple_hidden_layers_1.summary()
# Print activation functions
print("Activation Functions:")
for layer in cnn_model_with_multiple_hidden_layers_1.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# Train the model
start_time = time.time()
history = cnn_model_with_multiple_hidden_layers_1.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[tensorboard_callback])
end_time = time.time()
training_time_per_epoch = (end_time - start_time) / epochs
print("Training Time Per Epoch:", training_time_per_epoch, "seconds")

# Convert accuracy values to percentage
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# Plot accuracy versus epoch in percentage
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate the cnn_model_with_multiple_hidden_layers on the test data available in val directory dataset
test_loss, test_accuracy = cnn_model_with_multiple_hidden_layers_1.evaluate(test_ds)

# Plot accuracy in percentage
print("Test Accuracy:", test_accuracy * 100, "%")

# Predict labels for the test dataset
test_predictions = cnn_model_with_multiple_hidden_layers_1.predict(test_ds)
test_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# CNN with multiple hidden layer, RELU activation
batch_size = 32 # to run back propagation epoch
# img_height = 768
# img_width = 32
img_height = 390
img_width = 16
validation_split=0.2
seed=12081970
epochs = 10

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = len(class_names)
input_shape = (img_height,img_width, 3)

cnn_model_with_multiple_hidden_layers_2 = models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width,3)),
    layers.Conv2D(12, 3, padding='same', activation='relu'), # filters 12, kernel size 3
    layers.MaxPooling2D((2, 2),strides=(2, 2), padding='same'),
    layers.Conv2D(24, 3, padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'),
    layers.Conv2D(48, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
cnn_model_with_multiple_hidden_layers_2._name = 'CNN_with_Multiple_Hidden_Layers_RELU_Activation_with_Non_Original_Image_Training_2'

# Compile the model
cnn_model_with_multiple_hidden_layers_2.compile(optimizer='adam',
                                  loss='sparse_categorical_crossentropy',
                                  metrics=['accuracy'])
# Model details
cnn_model_with_multiple_hidden_layers_2.summary()
# Print activation functions
print("Activation Functions:")
for layer in cnn_model_with_multiple_hidden_layers_2.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# Train the model
start_time = time.time()
history = cnn_model_with_multiple_hidden_layers_2.fit(train_ds, validation_data=val_ds, epochs=epochs)
end_time = time.time()
training_time_per_epoch = (end_time - start_time) / epochs
print("Training Time Per Epoch:", training_time_per_epoch, "seconds")

# Convert accuracy values to percentage
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# Plot accuracy versus epoch in percentage
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate the cnn_model_with_multiple_hidden_layers on the test data available in val directory dataset
test_loss, test_accuracy = cnn_model_with_multiple_hidden_layers_2.evaluate(test_ds)

# Plot accuracy in percentage
print("Test Accuracy:", test_accuracy * 100, "%")

# Predict labels for the test dataset
test_predictions = cnn_model_with_multiple_hidden_layers_2.predict(test_ds)
test_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# CNN with multiple hidden layer, RELU activation with gradient descent data visualization
# For visualizing gradient descent data we will use TensorBoard

from tensorflow.keras.callbacks import TensorBoard
import datetime

# Let's define a log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Let's create TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

batch_size = 32 # to run back propagation epoch
# img_height = 768
# img_width = 32
img_height = 390
img_width = 16
validation_split=0.2
seed=12081970
epochs = 10

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = len(class_names)
input_shape = (img_height,img_width, 3)

cnn_model_with_multiple_hidden_layers_gdv_3 = models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width,3)),
    layers.Conv2D(12, 3, padding='same', activation='relu'), # filters 12, kernel size 3
    layers.MaxPooling2D((2, 2),strides=(2, 2), padding='same'),
    layers.Conv2D(24, 3, padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'),
    layers.Conv2D(48, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
cnn_model_with_multiple_hidden_layers_gdv_3._name = 'CNN_with_Multiple_Hidden_Layers_RELU_Activation_with_Non_Original_Image_Training_GDV_3'

# Compile the model
cnn_model_with_multiple_hidden_layers_gdv_3.compile(optimizer='adam',
                                  loss='sparse_categorical_crossentropy',
                                  metrics=['accuracy'])
# Model details
cnn_model_with_multiple_hidden_layers_gdv_3.summary()
# Print activation functions
print("Activation Functions:")
for layer in cnn_model_with_multiple_hidden_layers_gdv_3.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# Train the model
start_time = time.time()
history = cnn_model_with_multiple_hidden_layers_gdv_3.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[tensorboard_callback])
end_time = time.time()
training_time_per_epoch = (end_time - start_time) / epochs
print("Training Time Per Epoch:", training_time_per_epoch, "seconds")

# Convert accuracy values to percentage
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# Plot accuracy versus epoch in percentage
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate the cnn_model_with_multiple_hidden_layers on the test data available in val directory dataset
test_loss, test_accuracy = cnn_model_with_multiple_hidden_layers_gdv_3.evaluate(test_ds)

# Plot accuracy in percentage
print("Test Accuracy:", test_accuracy * 100, "%")

# Predict labels for the test dataset
test_predictions = cnn_model_with_multiple_hidden_layers_gdv_3.predict(test_ds)
test_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# CNN with multiple hidden layer, RELU activation, and non-original image size for training set with gdv

from tensorflow.keras.callbacks import TensorBoard
import datetime

# Let's define a log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Let's create TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# let's train with original image sizes
i_height = 768
i_width = 32
val_split = 0.2
seed=12081970
b_size = 32
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, val_split, i_width, i_height, b_size, seed)
num_classes = len(class_names)
input_shape = (i_height,i_width, 3)
epochs = 10

cnn_model_with_multiple_hidden_layers_4 = models.Sequential([
    layers.Rescaling(1./255, input_shape=input_shape),
    layers.Conv2D(12, 3, padding='same', activation='relu'), # filters 12, kernel size 3
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(24, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
cnn_model_with_multiple_hidden_layers_4._name = 'CNN_with_Multiple_Hidden_Layers_RELU_Activation_with_Original_Image_Size_Training_4'

# Compile the model
cnn_model_with_multiple_hidden_layers_4.compile(optimizer='adam',
                                  loss='sparse_categorical_crossentropy',
                                  metrics=['accuracy'])
# Model details
cnn_model_with_multiple_hidden_layers_4.summary()
# Print activation functions
print("Activation Functions:")
for layer in cnn_model_with_multiple_hidden_layers_4.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# Train the model
start_time = time.time()
history = cnn_model_with_multiple_hidden_layers_4.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[tensorboard_callback])
end_time = time.time()
training_time_per_epoch = (end_time - start_time) / epochs
print("Training Time Per Epoch:", training_time_per_epoch, "seconds")

# Convert accuracy values to percentage
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# Plot accuracy versus epoch in percentage
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate the cnn_model_with_multiple_hidden_layers on the test data available in val directory dataset
test_loss, test_accuracy = cnn_model_with_multiple_hidden_layers_4.evaluate(test_ds)

# Plot accuracy in percentage
print("Test Accuracy:", test_accuracy * 100, "%")

# Predict labels for the test dataset
test_predictions = cnn_model_with_multiple_hidden_layers_4.predict(test_ds)
test_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Pre-trained ResNet model with gradient descent data visualization

# install necessary modules
!pip install tensorflow==2.12.0
!pip install tensorflow-datasets==4.8.0

# imports for Resnet50
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image

# For visualizing gradient descent data we will use TensorBoard
from tensorflow.keras.callbacks import TensorBoard
import datetime
# Let's define a log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Let's create TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Hyper parameters
batch_size = 32 # to run back propagation epoch
img_height = 768
img_width = 32
# img_height = 390
# img_width = 16
validation_split=0.2
seed=12081970
epochs = 10

# prepare up train and validation data sets
train_ds, val_ds, test_ds, class_names = setup_data_pipeline(train_dir, val_dir, validation_split, batch_size, img_height, img_width, seed)
num_classes = 3
input_shape = (img_height,img_width, 3)

# Due to repeat of data for training, we need to set the number of steps manually
total_samples = len(list(train_ds))
train_steps_per_epoch = total_samples # this is batch size
val_steps_per_epoch = total_samples # for valaidation
print('Length of the train data set is ' + str(train_steps_per_epoch))
print('Length of the validation data set is ' + str(val_steps_per_epoch))

# Load ResNet50 pretrained model without the top classification layer
print('Configuring ResNet50 with input shape of ' + str(img_width) + 'x' + str(img_height))
base_model = ResNet50(weights='imagenet', include_top=False)
# Freeze the base model layers
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the full model
gpr_object_classify_resnet50_model_1 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
for layer in base_model.layers:
  layer.trainable = False
  gpr_object_classify_resnet50_model_1.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

gpr_object_classify_resnet50_model_1.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)
gpr_object_classify_resnet50_model_1._name = 'ResNet50_with_Original_Image_Training_GDV_1'

# adapted resnet50 model summary
gpr_object_classify_resnet50_model_1.summary()
print("Activation Functions:")
for layer in gpr_object_classify_resnet50_model_1.layers:
    if hasattr(layer, 'activation'):
        print(layer.name, "-", layer.activation.__name__)
    else:
        print(layer.name, "- No activation")

# Set up TensorBoard callback
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

# Train the model
# let's ensure not to run out of training data
train_ds = train_ds.repeat()
val_ds = val_ds.repeat()

print('Training the ResNet50 with training data...')
start_time = time.time()
history = gpr_object_classify_resnet50_model_1.fit(
    train_ds,
    steps_per_epoch=train_steps_per_epoch,
    validation_data = val_ds,
    validation_steps=val_steps_per_epoch,
    epochs = epochs,
    callbacks=[tensorboard_callback])
end_time = time.time()
training_time_per_epoch = (end_time - start_time) / epochs
print('Done - Training the ResNet50 with training data.')
print("Training Time Per Epoch:", training_time_per_epoch, "seconds")

# Convert accuracy values to percentage
train_accuracy_percentage = [acc * 100 for acc in history.history['accuracy']]
val_accuracy_percentage = [acc * 100 for acc in history.history['val_accuracy']]

# Plot accuracy versus epoch in percentage
plt.plot(train_accuracy_percentage, label='Training Accuracy', color='blue')
plt.plot(val_accuracy_percentage, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot training time per epoch
plt.plot(range(1, epochs + 1), [training_time_per_epoch] * epochs, label='Training Time per Epoch', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate the cnn_model_with_multiple_hidden_layers on the test data available in val directory dataset
test_loss, test_accuracy = gpr_object_classify_resnet50_model_1.evaluate(test_ds)

# Plot accuracy in percentage
print("Test Accuracy:", test_accuracy * 100, "%")

# Predict labels for the test dataset
test_predictions = gpr_object_classify_resnet50_model_1.predict(test_ds)
test_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
%tensorboard --logdir logs